In [ ]:
!pip install ultralytics -qq
!pip install pylabel -qq
!pip install lxml tqdm -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00


In [4]:
import zipfile
import os, random, cv2, json, shutil, yaml
import matplotlib.pyplot as plt
import ultralytics

from google.colab import drive
from ultralytics import YOLO
from pylabel import importer
from tqdm import tqdm
from collections import defaultdict

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# import os
# import json
# from collections import defaultdict
# from PIL import Image

# # ========== EDIT THESE PATHS ==========
# # JSONs
# train_json = "/content/drive/MyDrive/COCO2017/annotations/filtered_train2017.json"
# val_json   = "/content/drive/MyDrive/COCO2017/annotations/filtered_val2017.json"

# # lists (one filename per line, basename or path)
# train_list_path = "/content/drive/MyDrive/COCO_FILTERD/txt/train_images.txt"
# val_list_path   = "/content/drive/MyDrive/COCO_FILTERD/txt/val_images.txt"

# # image folders
# train_images_dir = "/content/drive/MyDrive/COCO_FILTERD/images/train"
# val_images_dir   = "/content/drive/MyDrive/COCO_FILTERD/images/val"

# # output label folders (will contain /labels subfolder)
# out_root_train = "/content/drive/MyDrive/COCO_FILTERD/labels/train"
# out_root_val   = "/content/drive/MyDrive/COCO_FILTERD/labels/val"

# # shared mapping output (we save it under a common parent)
# mapping_out_dir = "/content/drive/MyDrive/COCO_FILTERD/labels"  # will contain coco_to_yolo.json, yolo_to_coco.json, names.txt
# mapping_path = os.path.join(mapping_out_dir, "coco_to_yolo.json")
# # ======================================

# os.makedirs(out_root_train, exist_ok=True)
# os.makedirs(out_root_val, exist_ok=True)
# os.makedirs(mapping_out_dir, exist_ok=True)

# def load_list(path):
#     if not path or not os.path.exists(path):
#         return []
#     items = []
#     with open(path, "r") as f:
#         for line in f:
#             s = line.strip()
#             if s:
#                 items.append(os.path.basename(s))
#     return items

# def load_coco(path):
#     if not path or not os.path.exists(path):
#         return {"images": [], "annotations": [], "categories": []}
#     with open(path, "r") as f:
#         return json.load(f)

# def build_or_load_mapping(coco_list, mapping_path, mapping_out_dir):
#     # try load existing mapping
#     if os.path.exists(mapping_path):
#         try:
#             with open(mapping_path, "r") as f:
#                 coco_to_yolo = json.load(f)
#             print("Loaded existing mapping:", mapping_path)
#             return coco_to_yolo, None, None
#         except Exception as e:
#             print("Failed to load mapping, will rebuild. Error:", e)

#     # build mapping from union of categories across provided cocos
#     cats_union = {}
#     for coco in coco_list:
#         for c in coco.get("categories", []):
#             cats_union[c["id"]] = c["name"]

#     # sort by category id for stability
#     cats_sorted = sorted(cats_union.items(), key=lambda x: int(x[0]))

#     coco_to_yolo = {}
#     yolo_to_coco = {}
#     names = []
#     for new_idx, (cid, name) in enumerate(cats_sorted):
#         coco_to_yolo[str(cid)] = new_idx
#         yolo_to_coco[str(new_idx)] = int(cid)
#         names.append(name)

#     # save mapping + names
#     with open(os.path.join(mapping_out_dir, "coco_to_yolo.json"), "w") as f:
#         json.dump(coco_to_yolo, f, indent=2)
#     with open(os.path.join(mapping_out_dir, "yolo_to_coco.json"), "w") as f:
#         json.dump(yolo_to_coco, f, indent=2)
#     with open(os.path.join(mapping_out_dir, "names.txt"), "w") as f:
#         f.write("\n".join(names))

#     print(f"Built mapping with {len(names)} categories and saved under {mapping_out_dir}")
#     return coco_to_yolo, yolo_to_coco, names

# def generate_labels_for_list(coco, fname_to_id, anns_by_img, names_map, coco_to_yolo, images_dir, list_names, labels_out_dir, missing_json_list_path, missing_image_list_path):
#     os.makedirs(labels_out_dir, exist_ok=True)
#     written = 0
#     missing_in_json = []
#     missing_image_file = []

#     for fname in list_names:
#         base = os.path.splitext(fname)[0]
#         out_path = os.path.join(labels_out_dir, base + ".txt")

#         if fname not in fname_to_id:
#             missing_in_json.append(fname)
#             open(out_path, "w").close()
#             continue

#         img_id = fname_to_id[fname]
#         iminfo = coco["images_by_id"].get(img_id)
#         if iminfo is None:
#             missing_in_json.append(fname)
#             open(out_path, "w").close()
#             continue

#         W = iminfo.get("width")
#         H = iminfo.get("height")

#         # try to get size from disk if missing
#         if (W is None or H is None):
#             img_path = os.path.join(images_dir, fname)
#             if os.path.exists(img_path):
#                 try:
#                     with Image.open(img_path) as im:
#                         W, H = im.size
#                 except Exception:
#                     missing_image_file.append(fname)
#                     open(out_path, "w").close()
#                     continue
#             else:
#                 missing_image_file.append(fname)
#                 open(out_path, "w").close()
#                 continue

#         lines = []
#         for ann in anns_by_img.get(img_id, []):
#             x, y, w, h = ann["bbox"]
#             if w <= 0 or h <= 0:
#                 continue
#             coco_cat = ann["category_id"]
#             if str(coco_cat) in coco_to_yolo:
#                 cls = int(coco_to_yolo[str(coco_cat)])
#             else:
#                 cls = int(coco_cat)
#             cx = (x + w/2) / float(W)
#             cy = (y + h/2) / float(H)
#             wn = w / float(W)
#             hn = h / float(H)
#             # clamp
#             cx = max(0.0, min(1.0, cx))
#             cy = max(0.0, min(1.0, cy))
#             wn = max(0.0, min(1.0, wn))
#             hn = max(0.0, min(1.0, hn))
#             lines.append(f"{cls} {cx:.6f} {cy:.6f} {wn:.6f} {hn:.6f}")

#         with open(out_path, "w") as f:
#             f.write("\n".join(lines))
#         written += 1

#     # save missing lists
#     if missing_in_json and missing_json_list_path:
#         with open(missing_json_list_path, "w") as f:
#             for n in missing_in_json:
#                 f.write(n + "\n")
#     if missing_image_file and missing_image_list_path:
#         with open(missing_image_list_path, "w") as f:
#             for n in missing_image_file:
#                 f.write(n + "\n")

#     return written, missing_in_json, missing_image_file

# # -------- Load lists ----------
# train_names = load_list(train_list_path)
# val_names = load_list(val_list_path)

# print(f"Train list: {len(train_names)} entries (unique {len(set(train_names))})")
# print(f"Val list:   {len(val_names)} entries (unique {len(set(val_names))})")

# # -------- Load COCO JSONs ----------
# train_coco = load_coco(train_json)
# val_coco   = load_coco(val_json)

# # Build quick maps for train and val
# def prepare_coco_maps(coco):
#     images = coco.get("images", [])
#     anns = coco.get("annotations", [])
#     images_by_id = {im["id"]: im for im in images}
#     fname_to_id = {os.path.basename(im["file_name"]): im["id"] for im in images}
#     anns_by_img = defaultdict(list)
#     for ann in anns:
#         anns_by_img[ann["image_id"]].append(ann)
#     coco["images_by_id"] = images_by_id
#     coco["fname_to_id"] = fname_to_id
#     coco["anns_by_img"] = anns_by_img

# prepare_coco_maps(train_coco)
# prepare_coco_maps(val_coco)

# # -------- Build or load mapping from union of categories ----------
# coco_to_yolo, yolo_to_coco, names = build_or_load_mapping([train_coco, val_coco], mapping_path, mapping_out_dir)

# # if build_or_load returned only coco_to_yolo (when loaded), attempt to also build names from json(s)
# if names is None:
#     # try to collect category names from train then val
#     cats = train_coco.get("categories", []) or val_coco.get("categories", [])
#     cats_sorted = sorted(cats, key=lambda c: c["id"]) if cats else []
#     names = [c["name"] for c in cats_sorted] if cats_sorted else []

# # -------- Generate labels for train ----------
# written_train, missing_train_json, missing_train_image = generate_labels_for_list(
#     train_coco,
#     train_coco["fname_to_id"],
#     train_coco["anns_by_img"],
#     names,
#     coco_to_yolo,
#     train_images_dir,
#     train_names,
#     os.path.join(out_root_train, "labels"),
#     os.path.join(out_root_train, "missing_in_json.txt"),
#     os.path.join(out_root_train, "missing_image_files.txt")
# )

# print(f"\nTrain: Written {written_train} labels to {os.path.join(out_root_train, 'labels')}")
# if missing_train_json:
#     print(f"Train: {len(missing_train_json)} images in train_list not found in train.json (sample): {missing_train_json[:10]}")
# if missing_train_image:
#     print(f"Train: {len(missing_train_image)} train images missing on disk or unreadable (sample): {missing_train_image[:10]}")

# # -------- Generate labels for val ----------
# written_val, missing_val_json, missing_val_image = generate_labels_for_list(
#     val_coco,
#     val_coco["fname_to_id"],
#     val_coco["anns_by_img"],
#     names,
#     coco_to_yolo,
#     val_images_dir,
#     val_names,
#     os.path.join(out_root_val, "labels"),
#     os.path.join(out_root_val, "missing_in_json.txt"),
#     os.path.join(out_root_val, "missing_image_files.txt")
# )

# print(f"\nVal: Written {written_val} labels to {os.path.join(out_root_val, 'labels')}")
# if missing_val_json:
#     print(f"Val: {len(missing_val_json)} images in val_list not found in val.json (sample): {missing_val_json[:10]}")
# if missing_val_image:
#     print(f"Val: {len(missing_val_image)} val images missing on disk or unreadable (sample): {missing_val_image[:10]}")

# print("\nAll done.")


In [6]:
# output_path = "/content/drive/MyDrive/COCO_FILTERD/data.yaml"
# root_path   = "/content/drive/MyDrive/COCO_FILTERD"

# classes = ["person","chair","car","dining table","cup","bottle","bowl","handbag","truck","bench","backpack","book",
#            "cell phone","tv","couch","dog","knife","sports ball","traffic light","cat","umbrella","bus","bed","train",
#            "fork","spoon","laptop","motorcycle","bicycle","sandwich","banana","stop sign","orange","carrot","apple"
#            ]

# data_yaml = {
#     "path": root_path,
#     "train": "images/train",
#     "val": "images/val",
#     "nc": len(classes),
#     "names": classes
# }

# with open(output_path, "w") as f:
#     yaml.dump(data_yaml, f, sort_keys=False)

# print("data.yaml created at:", output_path)
# print("Number of classes =", len(classes))


In [8]:
root = "/content/drive/MyDrive/COCO_FILTERD"

train_images = os.path.join(root, "images/train")
train_labels = os.path.join(root, "labels/train")

val_images = os.path.join(root, "images/val")
val_labels = os.path.join(root, "labels/val")

def count_files(folder, exts):
    if not os.path.exists(folder):
        return 0
    return len([f for f in os.listdir(folder) if f.lower().endswith(exts)])

img_exts = (".jpg", ".jpeg", ".png")

print("===== TRAIN =====")
print("عدد صور train :", count_files(train_images, img_exts))
print("عدد labels train :", count_files(train_labels, (".txt",)))

print("\n===== VAL =====")
print("عدد صور val :", count_files(val_images, img_exts))
print("عدد labels val :", count_files(val_labels, (".txt",)))


===== TRAIN =====
عدد صور train : 59637
عدد labels train : 59637

===== VAL =====
عدد صور val : 4082
عدد labels val : 4082


In [7]:
DATA = "/content/drive/MyDrive/COCO_FILTERD/data.yaml"
MODEL = "yolov8n.pt"
SAVE_PATH = "/content/drive/MyDrive/COCO_FILTERD/saved_models/best_model.pt"
NAME = "glasses"

os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

model = YOLO(MODEL)

results = model.train(
    data=DATA,
    epochs=50,
    imgsz=640,
    batch=16,
    name=NAME
)


best_weight = f"runs/detect/{NAME}/weights/best.pt"

shutil.copy(best_weight, SAVE_PATH)

print("Model saved successfully at:", SAVE_PATH)